In [ ]:
import csv
import os
import tkinter as tk
from tkinter import messagebox, simpledialog, Toplevel, ttk
from datetime import datetime, timedelta

# --- FILE CONFIGURATION ---
BOOK_FILE = "Books.csv"

# Ensure CSV file exists with the correct headers
if not os.path.exists(BOOK_FILE):
    with open(BOOK_FILE, "w", newline="", encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([
            "Book_ID", "Title", "Author", "Year",
            "Genre", "Available", "Borrower", "Return_Date"
        ])

# --- DATA STRUCTURE 1: LINKED LIST ---
class BookNode:
    def __init__(self, book):
        self.book = book
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def insert(self, book):
        # Insert at the beginning (Head) - O(1) time complexity
        node = BookNode(book)
        node.next = self.head
        self.head = node

    def delete(self, book_id):
        current = self.head
        prev = None
        while current:
            if current.book["Book_ID"] == book_id:
                if prev:
                    prev.next = current.next
                else:
                    self.head = current.next
                return True
            prev = current
            current = current.next
        return False

    def to_list(self):
        # Convert Linked List to a standard Python list for display
        books = []
        current = self.head
        while current:
            books.append(current.book)
            current = current.next
        return books
    
    # Linear Search for partial matching (Title, Author, Genre)
    def search_keyword(self, keyword):
        results = []
        current = self.head
        keyword = keyword.lower()
        while current:
            b = current.book
            # Check if keyword exists in Title, Author, or Genre
            if (keyword in b['Title'].lower() or 
                keyword in b['Author'].lower() or 
                keyword in b['Genre'].lower()):
                results.append(b)
            current = current.next
        return results

# --- MAIN SYSTEM (GUI & LOGIC) ---
class LibrarySystem:
    def __init__(self, root):
        self.root = root
        self.root.title("Library Book Inventory System (DSA Project)")
        self.root.geometry("600x500") 
        
        # Initialize Data Structures
        self.book_list = LinkedList() # Linked List for dynamic listing
        self.book_table = {}          # Hash Table (Dictionary) for fast ID lookup

        self.load_books()
        self.main_menu()

    def load_books(self):
        # Reset data structures
        self.book_list = LinkedList()
        self.book_table = {}
        
        try:
            with open(BOOK_FILE, newline="", encoding='utf-8') as f:
                reader = csv.DictReader(f)
                for row in reader:
                    # Clean ID whitespace
                    row["Book_ID"] = row["Book_ID"].strip().upper()
                    
                    # Insert into BOTH structures (Sync)
                    self.book_list.insert(row)
                    self.book_table[row["Book_ID"]] = row
        except FileNotFoundError:
            pass

    def save_books(self):
        # Save data from Linked List back to CSV
        with open(BOOK_FILE, "w", newline="", encoding='utf-8') as f:
            fieldnames = [
                "Book_ID", "Title", "Author", "Year",
                "Genre", "Available", "Borrower", "Return_Date"
            ]
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            # Write all books found in the Linked List
            for book in self.book_list.to_list():
                writer.writerow(book)

    # --- USER INTERFACE (GUI) ---
    def main_menu(self):
        for widget in self.root.winfo_children():
            widget.destroy()

        # Header
        tk.Label(self.root, text="Library Inventory System", font=("Helvetica", 20, "bold")).pack(pady=20)
        
        # Button Frame for layout
        btn_frame = tk.Frame(self.root)
        btn_frame.pack(pady=10)

        # Operation Buttons
        tk.Button(btn_frame, text="🔍 Search Books", width=25, command=self.search_ui, bg="#e1f5fe").grid(row=0, column=0, padx=5, pady=5)
        tk.Button(btn_frame, text="📚 View All Books", width=25, command=self.view_books_ui, bg="#e1f5fe").grid(row=0, column=1, padx=5, pady=5)
        
        tk.Button(btn_frame, text="➕ Add New Book", width=25, command=self.add_book_ui, bg="#e8f5e9").grid(row=1, column=0, padx=5, pady=5)
        tk.Button(btn_frame, text="❌ Delete Book", width=25, command=self.delete_book_ui, bg="#ffebee").grid(row=1, column=1, padx=5, pady=5)

        tk.Button(btn_frame, text="📖 Borrow Book", width=25, command=self.borrow_book_ui, bg="#fff3e0").grid(row=2, column=0, padx=5, pady=5)
        tk.Button(btn_frame, text="↩️ Return Book", width=25, command=self.return_book_ui, bg="#fff3e0").grid(row=2, column=1, padx=5, pady=5)

        tk.Button(self.root, text="Exit Application", width=20, command=self.exit_system, bg="#bdbdbd").pack(pady=30)
        
        # Footer Credits
        tk.Label(self.root, text="Developed for BWS21103 Data Structure & Algorithms", font=("Arial", 8), fg="grey").pack(side=tk.BOTTOM, pady=10)

    # --- SEARCH FUNCTION (Hash Table + Linked List) ---
    def search_ui(self):
        query = simpledialog.askstring("Search", "Enter Title, Author, or Book ID:")
        if not query:
            return
        
        query = query.strip()
        results = []

        # 1. Try Hash Table first (Fast access O(1) if input is an ID)
        if query.upper() in self.book_table:
            results.append(self.book_table[query.upper()])
        else:
            # 2. If not an ID, use Linked List traversal (O(n) for keywords)
            results = self.book_list.search_keyword(query)

        self.show_results_window(results, f"Search Results for '{query}'")

    # --- VIEW FUNCTION (Table Display) ---
    def view_books_ui(self):
        all_books = self.book_list.to_list()
        # Sort by ID for cleaner display
        all_books.sort(key=lambda x: x["Book_ID"]) 
        self.show_results_window(all_books, "All Library Books")

    def show_results_window(self, books, title):
        window = Toplevel(self.root)
        window.title(title)
        window.geometry("900x400")

        # Using Treeview for a table-like display
        cols = ("ID", "Title", "Author", "Genre", "Year", "Available", "Borrower")
        tree = ttk.Treeview(window, columns=cols, show='headings')
        
        # Set headings and column widths
        tree.heading("ID", text="ID")
        tree.column("ID", width=50)
        tree.heading("Title", text="Title")
        tree.column("Title", width=250)
        tree.heading("Author", text="Author")
        tree.column("Author", width=150)
        tree.heading("Genre", text="Genre")
        tree.column("Genre", width=100)
        tree.heading("Year", text="Year")
        tree.column("Year", width=50)
        tree.heading("Available", text="Avail")
        tree.column("Available", width=50)
        tree.heading("Borrower", text="Borrower")
        tree.column("Borrower", width=100)

        # Insert data into the table
        for b in books:
            tree.insert("", "end", values=(
                b["Book_ID"], b["Title"], b["Author"], b["Genre"], 
                b["Year"], b["Available"], b["Borrower"]
            ))

        # Scrollbar configuration
        scrollbar = ttk.Scrollbar(window, orient="vertical", command=tree.yview)
        tree.configure(yscrollcommand=scrollbar.set)
        scrollbar.pack(side="right", fill="y")
        tree.pack(expand=True, fill="both")

    # --- ADD FUNCTION (Insert) ---
    def add_book_ui(self):
        add_win = Toplevel(self.root)
        add_win.title("Add New Book")
        add_win.geometry("300x400")

        fields = ["Book ID", "Title", "Author", "Year", "Genre"]
        entries = {}

        for i, field in enumerate(fields):
            tk.Label(add_win, text=field).pack(pady=2)
            entry = tk.Entry(add_win)
            entry.pack(pady=2)
            entries[field] = entry

        def submit_book():
            data = {f: entries[f].get().strip() for f in fields}
            
            # Simple validation
            if not data["Book ID"] or not data["Title"]:
                messagebox.showerror("Error", "ID and Title are required!")
                return
            
            # Check for Duplicate ID in Hash Table
            if data["Book ID"] in self.book_table:
                messagebox.showerror("Error", "Book ID already exists!")
                return

            # Default values
            new_book = {
                "Book_ID": data["Book ID"],
                "Title": data["Title"],
                "Author": data["Author"],
                "Year": data["Year"],
                "Genre": data["Genre"],
                "Available": "Yes",
                "Borrower": "",
                "Return_Date": ""
            }

            # Update BOTH Linked List and Hash Table
            self.book_list.insert(new_book)
            self.book_table[new_book["Book_ID"]] = new_book
            self.save_books()
            
            messagebox.showinfo("Success", "Book Added Successfully!")
            add_win.destroy()

        tk.Button(add_win, text="Save Book", command=submit_book, bg="#a5d6a7").pack(pady=20)

    # --- DELETE FUNCTION ---
    def delete_book_ui(self):
        book_id = simpledialog.askstring("Delete Book", "Enter Book ID to Delete:")
        if not book_id: return
        
        book_id = book_id.strip().upper()

        if book_id in self.book_table:
            confirm = messagebox.askyesno("Confirm", f"Are you sure you want to delete {book_id}?")
            if confirm:
                # Remove from Hash Table
                del self.book_table[book_id]
                # Remove from Linked List
                self.book_list.delete(book_id)
                
                self.save_books()
                messagebox.showinfo("Success", "Book deleted.")
        else:
            messagebox.showerror("Error", "Book ID not found.")

    # --- BORROW FUNCTION (Hash Table Access) ---
    def borrow_book_ui(self):
        book_id = simpledialog.askstring("Borrow", "Enter Book ID:")
        if not book_id: return
        book_id = book_id.strip().upper()

        # Direct Hash Table access (Fast)
        if book_id not in self.book_table:
            messagebox.showerror("Error", "Invalid Book ID.")
            return

        book = self.book_table[book_id]

        if book["Available"].lower() == "no":
            messagebox.showerror("Unavailable", f"Book currently borrowed by {book['Borrower']}")
            return

        borrower_name = simpledialog.askstring("Borrower", "Enter Student Name:")
        if borrower_name:
            # Update data (Since it's a reference, it updates everywhere)
            book["Available"] = "No"
            book["Borrower"] = borrower_name
            # Set return date to 7 days from now
            book["Return_Date"] = (datetime.now() + timedelta(days=7)).strftime("%Y-%m-%d")
            
            self.save_books()
            messagebox.showinfo("Success", f"Book borrowed until {book['Return_Date']}")

    # --- RETURN FUNCTION ---
    def return_book_ui(self):
        book_id = simpledialog.askstring("Return", "Enter Book ID:")
        if not book_id: return
        book_id = book_id.strip().upper()

        if book_id not in self.book_table:
            messagebox.showerror("Error", "Invalid Book ID.")
            return

        book = self.book_table[book_id]

        if book["Available"].lower() == "yes":
            messagebox.showinfo("Info", "This book is already in the library.")
            return

        # Fine Calculation
        try:
            due_date = datetime.strptime(book["Return_Date"], "%Y-%m-%d")
            overdue_days = (datetime.now() - due_date).days
            # Fine is RM 0.50 per day late
            fine = max(0, overdue_days * 0.50) 
        except:
            fine = 0

        msg = "Book Returned."
        if fine > 0:
            msg += f"\n\nLATE PENALTY: RM {fine:.2f}"

        # Reset status to Available
        book["Available"] = "Yes"
        book["Borrower"] = ""
        book["Return_Date"] = ""
        
        self.save_books()
        messagebox.showinfo("Success", msg)

    def exit_system(self):
        self.save_books()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = LibrarySystem(root)
    root.mainloop()